In [2]:
# Libraries to install if not done before, restart kernel after installation
#!pip install shap
#!pip install lime
#!pip install dice_ml

import pandas as pd
import numpy as np

# Preparación de datos
from sklearn.metrics import mutual_info_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import StandardScaler

# modelo elegido Random Forest
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

# análisis de rendimiento
from sklearn.metrics import accuracy_score, f1_score, auc, recall_score, precision_score
from sklearn.metrics import make_scorer, confusion_matrix
from sklearn.model_selection import GridSearchCV, KFold, cross_val_score

# XAI methods
from sklearn.inspection import permutation_importance
from sklearn.inspection import PartialDependenceDisplay
from sklearn.inspection import partial_dependence
import shap
import lime
import dice_ml as dice

# model evaluation
from sklearn.metrics import f1_score
from sklearn.metrics import make_scorer

# calibración
from sklearn.metrics import log_loss, brier_score_loss

# gráficos
import seaborn as sns
from matplotlib import pyplot as plt

# ignorar warnings
import warnings
warnings.filterwarnings("ignore")

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [ ]:
df = pd.read_csv("diabetes.csv")
df_copia = df.copy()
df

In [ ]:
new_values = {
    0: 'N', # Negativo
    1: 'P', # Positivo
}

df["Outcome"] = df["Outcome"].map(new_values) #Sustituyo los valores enteros por categóricos
df

In [ ]:
df.shape

In [ ]:
print(df.dtypes)

In [ ]:
df.isnull().sum()

In [ ]:
print(f"Nº de filas duplicadas: {df.duplicated().sum()}") #Duplicated devuelve True o False, sum cuenta cuantas veces hay True, es decir, no hay duplicados

categoricas = df.select_dtypes(include=['object']).columns.tolist()
numericas = df.select_dtypes(include=['float64', 'int64']).columns.tolist()
print(f"Columnas categóricas: {categoricas}")
print(f"Columnas numéricas: {numericas}")

Puede verse que en columnas como BloodPressure, BMI, Glucose, Insulin, SkinThicknes el valor mínimo es 0, esto es imposible, lo más probable es que se deba a haber sustituido valores nulls por 0, por lo que voy a eliminar todas las filas en las que algún valor de estas columnas sea  == 0.


In [ ]:
df.describe()

In [ ]:
df[df == 0].count()

In [ ]:
df_clean = df[(df['Insulin'] != 0) & (df['BMI'] != 0) & (df['Glucose'] != 0) & (df['BloodPressure'] != 0) & (df['SkinThickness'] != 0)]
df_clean

In [ ]:
df_clean.describe()

LIMPIEZA DE DATOS FINALIZADA

ANALISIS DE VARIABLES CATEGÓRICAS

Este dataframe, no posee variables categóricas más allá del Outcome que he convertido en categórica manualmente. Por lo que no puede hacerse el mismo estudio que se les va a hacer a las numéricas.

In [ ]:
sns.set(font_scale = 1)
sns.countplot(x="Outcome", data=df_clean).set(ylabel = "Data count"
            , xlabel = "Negativo                                   Positivo")
plt.yticks(ticks=range(0, df_clean['Outcome'].value_counts().max() + 50, 50))
plt.title('Target distribution')

ESTE ES ANTES DE HACER NINGUNA LIMPIEZA

In [ ]:
sns.set(font_scale = 1)
sns.countplot(x="Outcome", data=df_copia).set(ylabel = "Data count"
            , xlabel = "Positivo                                   Negativo")
plt.yticks(ticks=range(0, df_copia['Outcome'].value_counts().max() + 50, 50))
plt.title('Target distribution')

ANALISIS DE VARIABLES NUMERICAS

In [ ]:
# This visualization takes a while, set the 'if' as "True" to run it
if True:#False:
    colors = ["b","r"]
    plt.figure(figsize=(30,50))
    sns.set(font_scale = 2.5)

    ax = sns.pairplot(df_clean, hue='Outcome', palette=colors, height=3, aspect = 1.9,
                      kind="kde")
    ax.fig.suptitle('Pair Plots to show potential relations between numerical features (double-click to enlarge)',
                size=40, ha='center', y = 1.05)
    plt.show()

In [ ]:
matrix_corr = df_clean.drop(columns=['Outcome']).corr().round(3)

In [ ]:
plt.figure(figsize=(25,10))
sns.set(font_scale = 2)

sns.heatmap(matrix_corr,annot=True,linewidths=.5, cmap="Blues")
plt.title('Heatmap showing correlations between numerical data')
plt.show()

Aquí se hace un análisis de la importancia de cada variable numérica, como puede verse, la principal es la glucosa, seguida por la edad y la insulina.

In [ ]:
feature_scores = []

for col in numericas:
    auc = roc_auc_score(df_clean['Outcome'], df_clean[col])
    if auc < 0.5: # in case the feature is negatively correlated with the target
        auc = roc_auc_score(df_clean['Outcome'], -df_clean[col])
    feature_scores.append((col, auc))

columns = ['feature', 'ROC_AUC']
df_scores = pd.DataFrame(feature_scores, columns=columns)
df_scores.sort_values(by=['ROC_AUC'],ascending=False).reset_index(drop = True)

In [ ]:
new_values = {
    'N': 0, # Negativo
    'P': 1, # Positivo
}

df_clean["Outcome"] = df_clean["Outcome"].map(new_values) #Sustituyo los valores enteros por categóricos

df_select = df_clean.copy()

categoricas = df_select.select_dtypes(include=['object']).columns.tolist()
numericas = df_select.select_dtypes(include=['int64','float64']).columns.tolist()
numericas = numericas[:-1] #Quito el Outcome
print(f"Columnas categóricas: {categoricas}") #Este dataset no tiene columnas categóricas, por lo que no hará falta tratarlas
print(f"Columnas numéricas: {numericas}")

In [ ]:
# Separar en entrenamiento y validación y test
df_full_train, df_test = train_test_split(df_select, test_size=0.2, random_state=1)

# Separar entrenamiento y validación, que debe ser un 20% del 80%
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=1)

len(df_train), len(df_val), len(df_test)

In [ ]:
#Con esto como he eliminado filas, reinicio los indices
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)

y_train = df_train['Outcome'].values
y_test = df_test['Outcome'].values
y_val = df_val['Outcome'].values

VARIABLES NUMERICAS ENTRENAMIENTO

In [ ]:
scaler = StandardScaler()

# TRAIN
X_train = df_train[numericas].values
X_train = scaler.fit_transform(X_train)

# VAL
X_val = df_val[numericas].values
X_val = scaler.transform(X_val)

In [ ]:
X_train.shape

In [ ]:
X_val.shape

REGRESION LOGISTICA

In [ ]:
y_train.shape

In [ ]:
LR = LogisticRegression(random_state= 1) # default solver is 'lbfgs' and regularization 'C=1'

LR.fit(X_train, y_train)
y_pred = LR.predict_proba(X_val)[:, 1]
# the output is a matrix:
# left column is the results for neg ('0', no heart failure), right column is for pos ('1', heart failure)

# preliminary threshold to convert the raw score into a probability:
t = 0.5  # above this threshold the raw score becomes 1, below, is zero

In [ ]:
# preliminary performance metric
acc = accuracy_score(y_val, y_pred >= t)
# it compares the 0 or 1 in y_val with the False or True of y_pred>=t

print('thres', 'acc')
print('%.2f %.3f' % (t, acc))

In [ ]:
# parameters grid
LR_param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],
    # the smaller the C the stronger the regularization, default is C=1
    'max_iter': [50, 100, 200],
    # default is 100, max iterations taken for the solver to converge to the min
    'solver': ['lbfgs'],
    # the default solver
    'penalty': ['none', 'l2']
    # default is 'l2'
}

# metric
metric = make_scorer(f1_score)

# model definition
LR_grid = GridSearchCV(
            estimator = LogisticRegression(random_state= 1),
            param_grid = LR_param_grid,
            refit = True,
            verbose = 1,
            cv = 5,  # the default is 5 for a 5-fold cross validation
            scoring = metric
)

# train the model for grid search
LR_grid.fit(X_train,y_train)
LR_grid.best_params_

# summarize
print('Mean F1-score: %.3f' % LR_grid.best_score_)
print('Standar Deviation:', LR_grid.cv_results_['std_test_score'][LR_grid.best_index_].round(3))
print('Best Parameters: %s' % LR_grid.best_params_)

Elección del mejor modelo

In [ ]:
LR = LogisticRegression(random_state= 1, max_iter =50)
LR.fit(X_train, y_train)
y_pred = LR.predict_proba(X_val)[:, 1]

In [ ]:
# define the function that calculates the metrics for several thresholds

def tune_threshold(y_val, y_pred, number_of_thres):

    thresholds = np.linspace(0, 1, number_of_thres)
    metrics =[]

    for t in thresholds:
        acc = accuracy_score(y_val, y_pred >= t)
        pr  = precision_score(y_val, y_pred >= t,zero_division=0)
        rec = recall_score(y_val, y_pred >= t)
        f1  = f1_score(y_val, y_pred>=t)
        metrics.append((t, acc, pr, rec, f1))

    columns = ['threshold','accuracy','precision','recall','F1']
    df_metrics = pd.DataFrame(metrics, columns=columns)

    return df_metrics

In [ ]:
# apply the thresholds function

df_metrics = tune_threshold(y_val, y_pred, 21)
df_metrics.plot(x='threshold', y=['accuracy','precision','recall','F1'],
                title='model performance on the validation dataset',
                kind="line",  style=['+-', 'o-', 'x-', 'd-'], figsize=(20, 5));

print('the ROC AUC is', roc_auc_score(y_val, y_pred).round(3))

plt.show()

In [ ]:
df_metrics[df_metrics.F1.round(1) == max(df_metrics.F1.round(1))].round(3)

In [ ]:
def calc_val_metrics(model, X_train, y_train, X_val, y_val, y_pred, t):

    val_metrics = []

    # TRAIN
    y_pred = model.predict_proba(X_train)[:, 1]
    acc = accuracy_score(y_train, y_pred >= t)
    f1 = f1_score(y_train, y_pred>=t)
    rec = recall_score(y_train, y_pred >= t)
    auc = roc_auc_score(y_train, y_pred)
    print('For the training dataset:',
            'ACC:', round(acc,3),'F1:', round(f1,3), 'recall:', round(rec,3), 'ROC AUC:', round(auc,3))

    # VAL
    y_pred = model.predict_proba(X_val)[:, 1]
    val_acc = accuracy_score(y_val, y_pred >= t)
    val_f1 =  f1_score(y_val, y_pred>=t)
    val_rec = recall_score(y_val, y_pred >= t)
    val_auc = roc_auc_score(y_val, y_pred)

    val_metrics.append((val_acc, val_f1, val_rec, val_auc))

    print('For the validation dataset:',
        'ACC:', round(val_acc,3), 'F1:', round(val_f1,3), 'recall:', round(val_rec,3), 'ROC AUC:', round(val_auc,3))

    return val_metrics

In [ ]:
LR_val_metrics = calc_val_metrics(LR, X_train, y_train, X_val, y_val, y_pred, t=0.4)

In [ ]:
#Generate the confusion matrix
cf_matrix = confusion_matrix(y_val, y_pred >= t)

ax = sns.heatmap(cf_matrix, annot=True, cmap='Blues')

ax.set_title('Confusion Matrix\n');
ax.set_xlabel('\nPredicted Values')
ax.set_ylabel('Actual Values ');

ax.xaxis.set_ticklabels(['0','1'])
ax.yaxis.set_ticklabels(['0','1'])

plt.show()

RANDOM FOREST

In [ ]:
#%%timeit  # this cells takes some seconds to run

# parameters
n_estimators = [50, 100, 150]
max_depth = [5, 10]
min_samples_leaf = [5, 10]

RF_param_grid = dict(max_depth = max_depth, min_samples_leaf = min_samples_leaf, n_estimators = n_estimators)

# model
RF = RandomForestClassifier(max_depth = max_depth,
                            min_samples_leaf = min_samples_leaf,
                            n_estimators = n_estimators,
                            random_state = 1)

# metric
metric = make_scorer(f1_score)

# grid
RF_grid = GridSearchCV(
        estimator = RF,
        param_grid = RF_param_grid,
        scoring = metric,
        verbose =1)

# train the model
RF_grid_results = RF_grid.fit(X_train, y_train)

# summarize
print('Mean F1-score: %.3f' % RF_grid.best_score_)
print('Standar Deviation:', RF_grid.cv_results_['std_test_score'][RF_grid.best_index_].round(3))
print('Best Parameters: %s' % RF_grid.best_params_)

In [ ]:
# let us extract the best random forest and apply it to the validation set

best_RF = RF_grid_results.best_estimator_
y_pred = best_RF.predict_proba(X_val)[:, 1]

In [ ]:
# apply the thresholds function

df_metrics = tune_threshold(y_val, y_pred, 41)
df_metrics.plot(x='threshold', y=['accuracy','precision','recall','F1'],
                title='model performance on the validation dataset',
                kind="line",  style=['+-', 'o-', 'x-', 'd-'], figsize=(20, 5));

print('the ROC AUC is', roc_auc_score(y_val, y_pred).round(3))

plt.show()

In [ ]:
df_metrics[df_metrics.F1.round(3) == max(df_metrics.F1.round(3))].round(3)

In [ ]:
t = 0.4
RF_val_metrics = calc_val_metrics(best_RF, X_train, y_train, X_val, y_val, y_pred, t=0.4)

In [ ]:
t=0.4
cf_matrix = confusion_matrix(y_val, y_pred >= t)

ax = sns.heatmap(cf_matrix, annot=True, cmap='Blues')

ax.set_title('Confusion Matrix\n');
ax.set_xlabel('\nPredicted Values')
ax.set_ylabel('Actual Values ');

ax.xaxis.set_ticklabels(['0','1'])
ax.yaxis.set_ticklabels(['0','1'])

plt.show()

EVALUACION

In [ ]:
print('Model performance on the validation dataset:')

df_metrics_metric = pd.DataFrame(['acc', 'F1', 'recall', 'ROC_AUC'])
df_metrics_LR = pd.DataFrame(LR_val_metrics).round(3).T
df_metrics_RF = pd.DataFrame(RF_val_metrics).round(3).T

df_final_val_metrics = pd.concat([df_metrics_metric, df_metrics_LR,
                              df_metrics_RF], axis=1)
df_final_val_metrics.columns = ['metric', 'LR', 'RF']
df_final_val_metrics.set_index('metric')

In [ ]:
models = []

# we programatically tuned the main parameters of these models:
models.append(('LR', LogisticRegression(max_iter = 50, random_state = 1)))

models.append(('RF', RandomForestClassifier(max_depth = 10,
                                            min_samples_leaf = 5,
                                            n_estimators =  50, random_state = 1)))

results = []
names = []
metric = make_scorer(f1_score) # try recall_score if you like

print("The mean and std of F1-score for the 10-folds cross-validation on the training set:")
print()

for name, model in models:
    kfold = KFold(n_splits = 10, shuffle = True)
    cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring= metric)
    results.append(cv_results)
    names.append(name)
    print(name, cv_results.mean().round(3), '+-', cv_results.std().round(3))

TEST

In [ ]:
# TRAIN
df_full_train = df_full_train.reset_index(drop=True) # reset index after splitting shuffling
y_full_train = df_full_train['Outcome'].values

# 2. scale the numerical features ---------------------------------------------------

X_full_train = df_full_train[numericas].values
X_full_train = scaler.fit_transform(X_full_train)

In [ ]:
# TEST
df_test = df_test.reset_index(drop=True) # reset index after splitting shuffling
y_test = df_test['Outcome'].values

del df_test['Outcome'] # remove target


# 2. scale the numerical features --------------------------------------------

X_test = df_test[numericas].values
print(X_test)
X_test = scaler.transform(X_test)

In [ ]:
# train and apply the final model:

RF = RandomForestClassifier(max_depth = 10, min_samples_leaf = 5,
                            n_estimators =  50, random_state = 1)

model = RF.fit(X_full_train, y_full_train)
y_pred = RF.predict_proba(X_test)[:, 1]

t = 0.4

acc = accuracy_score(y_test, y_pred >= t)
rec = recall_score(y_test, y_pred >= t)
f1  = f1_score(y_test, y_pred >= t)
auc = roc_auc_score(y_test, y_pred)
print('For the test dataset:',
      'ACC:', round(acc,3), 'F1:', round(f1,3),
      'recall:', round(rec,3),'ROC AUC:', round(auc,3))

In [ ]:
RF_val_metrics = calc_val_metrics(best_RF, X_train, y_train, X_val, y_val, y_pred, t=0.4)

In [ ]:
cf_matrix = confusion_matrix(y_test, y_pred >= t)

ax = sns.heatmap(cf_matrix, annot=True, cmap='Blues')

ax.set_title('Confusion Matrix\n');
ax.set_xlabel('\nPredicted Values')
ax.set_ylabel('Actual Values ');

ax.xaxis.set_ticklabels(['0','1'])
ax.yaxis.set_ticklabels(['0','1'])

##plt.figure(figsize=(7,5))
plt.rcParams['figure.figsize'] = [7, 5]  # re-run this cell to get the correct figure size

plt.show()

In [ ]:
new_patient = {
    'Pregnancies': 1,
    'Glucose': 0,
    'BloodPressure': 0,
    'SkinThickness': 1,
    'Insulin': 0,
    'BMI': 0,
    'DiabetesPedigreeFuntion': 3,
    'Age': 34
 }

In [ ]:
X = np.array(list(new_patient.values())).reshape(1, -1)
print(X)
y_pred = RF.predict_proba(X)[:, 1]
print('Model application to new patient:')
print()
print("The patient raw score of suffering a hear failure is:", y_pred[0].round(2))
print()
print("With t =", t, "as the decision threshold, is there a risk of suffering a heart failure?", y_pred[0] >= t)

IMPORTANCIA DE CADA COLUMNA

In [ ]:
metric = make_scorer(f1_score)

result = permutation_importance(RF, X_test, y_test, scoring = metric,
                                n_repeats = 10, random_state = 1)
RF_per_importances = pd.Series(result.importances_mean, index=feature_names)

In [ ]:
fig, ax = plt.subplots(figsize = (16, 9))
RF_per_importances.plot.bar(yerr=result.importances_std, ax = ax,fontsize = 20)
ax.set_title("Feature importance by permutation importance on the test set",fontsize = 20)
ax.set_ylabel("Mean F1-score decrease",fontsize = 20)
plt.show()